In [10]:
%load_ext autoreload

%autoreload 2

#from vae_dist.core.e3VAE import E3VAE
from vae_dist.core.training import train
from vae_dist.dataset.dataset import FieldDataset
import torch 
from e3nn.io import CartesianTensor, SphericalTensor
from e3nn import o3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = "../../data/cpet/"
# load model to gpu
dataset_vanilla = FieldDataset(
    root, 
    transform=None, 
    augmentation=None, 
    device=device
    )



Largest value in dataset:  0.9999999993409996
Smallest value in dataset:  0.0


In [13]:
data = dataset_vanilla[0]
print(data.shape[3])
data_reshape = data.reshape(1,3,data.shape[1], data.shape[2], data.shape[3])
#2x0e+1x1e+3x2e+1x3e+1x4e

#x = CartesianTensor("3x0e")
#sphere = x.from_cartesian(data_reshape)


x = CartesianTensor("ijk=-jik=-ikj")
print(x)
x.from_cartesian(torch.ones(1,3, 21, 21, 21))

21
1x0o


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x9261 and 27x1)

In [5]:
x = CartesianTensor("1e")
sphere = x.from_cartesian(torch.ones(3, 3, 3, 3))

"""from e3nn.nn.models.v2103.gate_points_networks import SimpleNetwork
net = SimpleNetwork(
    irreps_in="3x0e + 2x1o",
    irreps_out="1x1o",
    max_radius=2.0,
    num_neighbors=3.0,
    num_nodes=5.0
)"""

'from e3nn.nn.models.v2103.gate_points_networks import SimpleNetwork\nnet = SimpleNetwork(\n    irreps_in="3x0e + 2x1o",\n    irreps_out="1x1o",\n    max_radius=2.0,\n    num_neighbors=3.0,\n    num_nodes=5.0\n)'

In [6]:
#from e3nn import o3

test = dataset_vanilla[0]
test = test.reshape(21, 21, 21)
#test = torch.einsum('tixyz->txyzi', test)
print(test.shape)
#test = torch.einsum('icxyz->icxyz', test)
x = CartesianTensor("3e")
#sphere = x.from_cartesian(test)

#x = CartesianTensor('aijkl=ajikl=aijlk')
#print(x)
sphere = x.from_cartesian(test)

#analogous 
#nn.conv3d(in_channels=4, out_channels=4, kernel_size=5, stride=1) 
#e3nn.image.Convolution(Rs_in=[(4, 0)], Rs_out=[(4, 0)], size=5, stride=1)

RuntimeError: shape '[21, 21, 21]' is invalid for input of size 27783

In [ ]:
from e3nn.o3 import Irrep, Irreps
Irrep("1e")

1e

In [15]:
from functools import partial
import torch, math
from torch import nn
from e3nn.nn import BatchNorm, Gate, Dropout
from e3nn.nn.models.v2103.voxel_convolution import LowPassFilter
from e3nn.o3 import Irreps, Linear, FullyConnectedTensorProduct, Linear
from e3nn.math import soft_unit_step, soft_one_hot_linspace
from e3nn import o3


class Convolution(torch.nn.Module):
    r"""convolution on voxels
    Parameters
    ----------
    irreps_in : `Irreps`
        input irreps
    irreps_out : `Irreps`
        output irreps
    irreps_sh : `Irreps`
        set typically to ``o3.Irreps.spherical_harmonics(lmax)``
    diameter : float
        diameter of the filter in physical units
    num_radial_basis : int
        number of radial basis functions
    steps : tuple of float
        size of the pixel in physical units
    """
    def __init__(self, irreps_in, irreps_out, irreps_sh, diameter, num_radial_basis, steps=(1.0, 1.0, 1.0),cutoff=True, **kwargs):
        super().__init__()

        self.irreps_in = o3.Irreps(irreps_in)
        self.irreps_out = o3.Irreps(irreps_out)
        self.irreps_sh = o3.Irreps(irreps_sh)

        self.num_radial_basis = num_radial_basis


        # self-connection
        self.sc = Linear(self.irreps_in, self.irreps_out)

        # connection with neighbors
        r = diameter / 2

        s = math.floor(r / steps[0])
        x = torch.arange(-s, s + 1.0) * steps[0]

        s = math.floor(r / steps[1])
        y = torch.arange(-s, s + 1.0) * steps[1]

        s = math.floor(r / steps[2])
        z = torch.arange(-s, s + 1.0) * steps[2]

        lattice = torch.stack(torch.meshgrid(x, y, z), dim=-1)  # [x, y, z, R^3]
        self.register_buffer('lattice', lattice)

        if 'padding' not in kwargs:
            kwargs['padding'] = tuple(s // 2 for s in lattice.shape[:3])
        self.kwargs = kwargs

        emb = soft_one_hot_linspace(
            x=lattice.norm(dim=-1),
            start=0.0,
            end=r,
            number=self.num_radial_basis,
            basis='smooth_finite',
            cutoff=cutoff,
        )
        self.register_buffer('emb', emb)

        sh = o3.spherical_harmonics(
            l=self.irreps_sh,
            x=lattice,
            normalize=True,
            normalization='component'
        )  # [x, y, z, irreps_sh.dim]
        self.register_buffer('sh', sh)

        self.tp = FullyConnectedTensorProduct(self.irreps_in, self.irreps_sh, self.irreps_out, shared_weights=False,compile_right=True)

        self.weight = torch.nn.Parameter(torch.randn(self.num_radial_basis, self.tp.weight_numel))

    def kernel(self):
        weight = self.emb @ self.weight
        weight = weight / (self.sh.shape[0] * self.sh.shape[1] * self.sh.shape[2])
        kernel = self.tp.right(self.sh, weight)  # [x, y, z, irreps_in.dim, irreps_out.dim]
        kernel = torch.einsum('xyzio->oixyz', kernel)
        return kernel

    def forward(self, x):
        r"""
        Parameters
        ----------
        x : `torch.Tensor`
            tensor of shape ``(batch, irreps_in.dim, x, y, z)``
        Returns
        -------
        `torch.Tensor`
            tensor of shape ``(batch, irreps_out.dim, x, y, z)``
        """
        sc = self.sc(x.transpose(1, 4)).transpose(1, 4)

        return sc + torch.nn.functional.conv3d(x, self.kernel(), **self.kwargs)


class ConvolutionVoxel(torch.nn.Module):
    r"""convolution on voxels
    Parameters
    ----------
    irreps_in : `Irreps`
    irreps_out : `Irreps`
    irreps_sh : `Irreps`
        set typically to ``o3.Irreps.spherical_harmonics(lmax)``
    size : int
    steps : tuple of int
    """
    def __init__(self, irreps_in, irreps_out, irreps_sh, size, steps=(1, 1, 1),cutoff=True, **kwargs):
        super().__init__()

        self.irreps_in = o3.Irreps(irreps_in)
        self.irreps_out = o3.Irreps(irreps_out)
        self.irreps_sh = o3.Irreps(irreps_sh)
        self.size = size
        self.num_rbfs = self.size

        if 'padding' not in kwargs:
            kwargs['padding'] = self.size // 2
        self.kwargs = kwargs

        # self-connection
        self.sc = Linear(self.irreps_in, self.irreps_out)

        # connection with neighbors
        r = torch.linspace(-1, 1, self.size)
        x = r * steps[0] / min(steps)
        x = x[x.abs() <= 1]
        y = r * steps[1] / min(steps)
        y = y[y.abs() <= 1]
        z = r * steps[2] / min(steps)
        z = z[z.abs() <= 1]
        lattice = torch.stack(torch.meshgrid(x, y, z), dim=-1)  # [x, y, z, R^3]
        emb = soft_one_hot_linspace(
            x=lattice.norm(dim=-1),
            start=0.0,
            end=1.0,
            number=self.num_rbfs,
            basis='smooth_finite',
            cutoff=cutoff,
        )
        self.register_buffer('emb', emb)

        sh = o3.spherical_harmonics(self.irreps_sh, lattice, True, 'component')  # [x, y, z, irreps_sh.dim]
        self.register_buffer('sh', sh)

        self.tp = FullyConnectedTensorProduct(self.irreps_in, self.irreps_sh, self.irreps_out, shared_weights=False)

        self.weight = torch.nn.Parameter(torch.randn(self.num_rbfs, self.tp.weight_numel))

    def forward(self, x):
        r"""
        Parameters
        ----------
        x : `torch.Tensor`
            tensor of shape ``(batch, irreps_in.dim, x, y, z)``
        Returns
        -------
        `torch.Tensor`
            tensor of shape ``(batch, irreps_out.dim, x, y, z)``
        """
        sc = self.sc(x.transpose(1, 4)).transpose(1, 4)

        weight = self.emb @ self.weight
        weight = weight / (self.size ** (3/2))
        kernel = self.tp.right(self.sh, weight)  # [x, y, z, irreps_in.dim, irreps_out.dim]
        kernel = torch.einsum('xyzio->oixyz', kernel)
        return sc + 0.1 * torch.nn.functional.conv3d(x, kernel, **self.kwargs)


class LowPassFilter(torch.nn.Module):
    def __init__(self, scale, stride=1, transposed=False, steps=(1, 1, 1)):
        super().__init__()

        sigma = 0.5 * (scale ** 2 - 1)**0.5

        size = int(1 + 2 * 2.5 * sigma)
        if size % 2 == 0:
            size += 1

        r = torch.linspace(-1, 1, size)
        x = r * steps[0] / min(steps)
        x = x[x.abs() <= 1]
        y = r * steps[1] / min(steps)
        y = y[y.abs() <= 1]
        z = r * steps[2] / min(steps)
        z = z[z.abs() <= 1]
        lattice = torch.stack(torch.meshgrid(x, y, z), dim=-1)  # [x, y, z, R^3]
        lattice = (size // 2) * lattice

        kernel = torch.exp(-lattice.norm(dim=-1).pow(2) / (2 * sigma**2))
        kernel = kernel / kernel.sum()
        if transposed:
            kernel = kernel * stride**3
        kernel = kernel[None, None]
        self.register_buffer('kernel', kernel)

        self.scale = scale
        self.stride = stride
        self.size = size
        self.transposed = transposed

    def forward(self, image):
        """
        Parameters
        ----------
        image : `torch.Tensor`
            tensor of shape ``(..., x, y, z)``
        Returns
        -------
        `torch.Tensor`
            tensor of shape ``(..., x, y, z)``
        """
        if self.scale <= 1:
            assert self.stride == 1
            return image

        out = image
        out = out.reshape(-1, 1, *out.shape[-3:])
        if self.transposed:
            out = torch.nn.functional.conv_transpose3d(out, self.kernel, padding=self.size // 2, stride=self.stride)
        else:
            out = torch.nn.functional.conv3d(out, self.kernel, padding=self.size // 2, stride=self.stride)
        out = out.reshape(*image.shape[:-3], *out.shape[-3:])
        return out







In [17]:

def test():
    conv = Convolution(
        "1e", " 2o", 
        o3.Irreps.spherical_harmonics(lmax=3), 
        diameter=5, num_radial_basis=4, steps=(1, 1, 1))

    x = torch.randn(1, 3, 21, 21, 21)
    #pad_x = torch.zeros((1, 4, 21, 21, 21), device=x.device, dtype=x.dtype)
    #pad_x[:, :x.size(1), :, :, :] = x

    #x = torch.randn(1, 3, 21, 21, 21)
    fi = LowPassFilter(2.0)
    #print(fi(pad_x).shape)
    print(conv(x).shape)
    #fi(x)

test()

torch.Size([1, 5, 21, 21, 21])


/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809535/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
x = torch.randn(1, 3, 21, 21, 21)
pad_x = torch.zeros((1, 4, 21, 21, 21), device=x.device, dtype=x.dtype)
pad_x[:, :x.size(1), :, :, :] = x

In [ ]:
print(x.shape)
print(pad_x.shape)

torch.Size([1, 3, 21, 21, 21])
torch.Size([1, 4, 21, 21, 21])
